In [20]:
def get_actual_ids_first_k(actual_sorted_ids, k):
    return [actual_sorted_ids[id] for id in range(len(actual_sorted_ids)) if id < k]
print(get_actual_ids_first_k([43,53,34,3,4,5],3))

[43, 53, 34]


In [21]:
import struct
import os
from typing import Dict, List, Annotated
import numpy as np
from sklearn.cluster import MiniBatchKMeans


class VecDB:

    def __init__(self, file_path = "PATH_DB_10K", new_db = True) -> None:
        self._file_path = file_path
        if new_db:
            # just open new file to delete the old one
            #with open(self._file_path +"/data_points.npy", "w") as fout:
                # if you need to add any head to the file
                #pass
            pass

    
    def insert_records(self, rows: List[Dict[int, Annotated[List[float], 70]]]):
        #Features Vectors
        embedings_rows=[]
        self.db_size=len(rows)
        for row in rows:
            _, embed = row["id"], row["embed"]
            embedings_rows.append(embed)
        # Create the directory if it doesn't exist
        os.makedirs("./" + self._file_path , exist_ok=True)
          
        #Creating 1-Level Indexing File        
        self._build_index(embedings_rows)
        
        
    def retrive(self , query: Annotated[List[float], 5], top_k = 10):
        # Approx. Worst Ram Usage for 20M data Points = 289.001 MB = 20M *(0.03% * (70features * 4Bytes + 4Bytes + 2* (2 score tuple * 4Bytes ))) + 10K * ( 70features * 4Bytes + 2* (2 score tuple * 4Bytes )+ (2 index tuple * 4Bytes ) ) = 303.04 * 10^6 Bytes        
        # Loading centroids do use it in calculating cosine simularity with query vector
        centriods = np.load("./" + self._file_path + "/Cluster_Centriods.npy",allow_pickle=True)
        #print("centriods Dimention is : ", len(centriods)," centriod of ",len(centriods[0])," features.")
        scores = []
        for id in range(len(centriods)):
            # Access the current centroid using its index
            centriod = centriods[id]
            #calculating cosine simularity
            score = self._cal_score(query, centriod)
            scores.append((score, id))
        sorted_scores= sorted(scores, reverse=True)            
        # here we assume that if two rows have the same score, return the lowest ID
        first_index = sorted_scores[0][1] #First closest Centriod index
        ########################TODO : comment this code
        ##print ('First closest Centriod is : ', first_index)
        ########################
        # Loading centroids ids points use it in calculating cosine simularity with query vector
        second_index = sorted_scores[1][1] #Second closest Centriod index
        ########################TODO : comment this code
        #print ('Second closest Centriod is : ', second_index)
        ########################
        # Loading centroids ids points use it in calculating cosine simularity with query vector
        third_index = sorted_scores[2][1] #Third closest Centriod index
        ########################TODO : comment this code
        #print ('Third closest Centriod is : ', third_index)
        ########################
        # Loading centroids ids points use it in calculating cosine simularity with query vector                
        cluster_indexes = np.load("./" + self._file_path + "/Cluster_Indexes.npy",allow_pickle=True)
        
        first_cluster_start_index = cluster_indexes[first_index][0]
        first_cluster_end_index = cluster_indexes[first_index][1]
        #print("first_cluster_start_index is : ",first_cluster_start_index," & first_cluster_end_index ",first_cluster_end_index)
        
        second_cluster_start_index = cluster_indexes[second_index][0]
        second_cluster_end_index = cluster_indexes[second_index][1]
        #print("second_cluster_start_index is : ",second_cluster_start_index," & second_cluster_end_index ",second_cluster_end_index)
        
        third_cluster_start_index = cluster_indexes[third_index][0]
        third_cluster_end_index = cluster_indexes[third_index][1]
        #print("third_cluster_start_index is : ",third_cluster_start_index," & third_cluster_end_index ",third_cluster_end_index)
        
        # Loading chosen points indexes use it in calculating cosine simularity with query vector        
        indexes =[]
        data_points_indexes = np.lib.format.open_memmap("./" + self._file_path + "/data_points_indexes"+".npy", mode='r', dtype='int') 

        first_cluster_data_points_indexes = data_points_indexes[first_cluster_start_index:first_cluster_end_index]
        indexes += first_cluster_data_points_indexes.tolist()
        
        second_cluster_data_points_indexes = data_points_indexes[second_cluster_start_index:second_cluster_end_index]
        indexes += second_cluster_data_points_indexes.tolist()   
        
        third_cluster_data_points_indexes = data_points_indexes[third_cluster_start_index:third_cluster_end_index]
        indexes += third_cluster_data_points_indexes.tolist() 
        
        #print("indexes (First 10) = ", indexes[:10])
        #print("indexes Dimention is : ", len(indexes)," index")
        
        # Loading chosen points use it in calculating cosine simularity with query vector
        
        rows =[]
        data_points = np.lib.format.open_memmap("./" + self._file_path + "/data_points"+".npy", mode='r', dtype='float32' , shape=(len(indexes),70)) 

        first_cluster_data_points = data_points[first_cluster_start_index:first_cluster_end_index]
        rows += first_cluster_data_points.tolist() 
        
        second_cluster_data_points = data_points[second_cluster_start_index:second_cluster_end_index]
        rows += second_cluster_data_points.tolist()   
        
        third_cluster_data_points = data_points[third_cluster_start_index:third_cluster_end_index]
        rows += third_cluster_data_points.tolist()           
        
        #print("sorted rows Data Dimention is : ", len(rows)," point of ",len(rows[0])," features.")
        #print("sorted rows Data (First 10 row)(First 3 feature) = ", [row[:3] for row in rows[:10]])

        final_scores = []
        for i in range(len(rows)):
            #calculating cosine simularity
            score = self._cal_score(query, rows[i])
            final_scores.append((score, indexes[i]))

        final_indexes = sorted(final_scores  , reverse=True)[:top_k] #closest points
        ########################TODO : comment this code
        #print ('closest Points_indexes is : ', [s[1] for s in final_indexes])
        #print ('closest Points score is : ', [s[0] for s in final_indexes])
        ########################            
        return [s[1] for s in final_indexes]        


    # Calculate Cosine Similarity 
    def _cal_score(self, vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        cosine_similarity = dot_product / (norm_vec1 * norm_vec2)
        return cosine_similarity


     
     

    def _build_index(self,rows: List[Annotated[List[float], 70]]):
        #Approx. Worst Ram Usage for 20M data Points = 10.657 GB = 20M *(70features * 4Bytes + 70features * 4Bytes + 4Bytes + 4Bytes + 4Bytes) + 10K * ( 70features * 4Bytes + 2indexes * 4Bytes + 4Bytes ) = 1.144292 * 10^10 Bytes
        #print("rows_points Dimention is : ", len(rows)," point of ",len(rows[0])," features.")
        #print("First 10 row_points (First 3 feature): ", [row[:3] for row in rows[:10]])


        # num_Clusters = Data Size / 2000 #Every Cluster has 2000 record 
        # batch_Size = By sense
        
        batch_Size=100
        n_Clusters=10
        if len(rows)== 10000: #10K
            batch_Size= 2000
            n_Clusters= 10
        elif len(rows)== 100000: #100K
            batch_Size= 20000
            n_Clusters= 50
        elif len(rows)== 1000000: #1M
            batch_Size= 100000
            n_Clusters= 500
        elif len(rows)== 5000000: #5M
            batch_Size= 1000000
            n_Clusters= 2500  
        elif len(rows)== 10000000:#10M
            batch_Size= 1000000
            n_Clusters= 5000
        elif len(rows)== 15000000: #15M
            batch_Size= 1000000
            n_Clusters= 7500 
        elif len(rows)== 20000000: #20M
            batch_Size= 1000000
            n_Clusters= 10000 
            
        batch_begin = 0
        ##Begin of learning Phase
        kmeans = MiniBatchKMeans(n_clusters = n_Clusters, batch_size = batch_Size, n_init=1)
        counter = len(rows)//batch_Size -1 # For Loaping over whole Data 
        
        while counter >= 0: #Loaping over whole Data using small batches steps 
            #Update k means estimate on a single mini-batch X.
            kmeans = kmeans.partial_fit(rows[batch_begin:batch_begin+batch_Size])
            batch_begin+=batch_Size
            counter-=1
            
        #Compute cluster centers and predict cluster index for each sample.    
        labels = kmeans.predict(rows)
        #Rearrange Data samples in List of Centiode each one has it's own points ids.
        arranged_data_samples = [[] for _ in range(n_Clusters)]
        #To git size of each Cluster
        clusters_size = [0 for _ in range(n_Clusters)]
        for index in range(len(rows)):
                arranged_data_samples[labels[index]].append(index)
                clusters_size[labels[index]] += 1 # increamnt
        #for i in range(n_Clusters):        
            ##print("Cluster [",i,"] : size of points: ", clusters_size[i])
            ##print("Cluster [",i,"] : First 5 points ids: ", arranged_data_samples[i][:5])

        #ReSort Data samples in List of indexes based on Clusters's index arrange from centiod 0 to #n_Clusters each one has it's own points ids.
        sorted_data_sample_indexes =[]  
        sorted_data_sample =[]  
        for cluster in arranged_data_samples:
                for index in cluster:
                    sorted_data_sample.append(rows[index])   #List of Float Embeddings 70 Feature Vectors                     
                    sorted_data_sample_indexes.append(index) #List of Int IDs      
        #print("First 10 sorted points (First 3 feature): ", [row[:3] for row in sorted_data_sample[:10]])
        #print("First 10 sorted points ids: ", sorted_data_sample_indexes[:10])                    
        clusters_start_end_indexes = [[0,0] for _ in range(n_Clusters)] # To read Embeddings 70 Feature Vectors List
        start_index = 0 #num_elemnts
        end_index = 0 #num_elemnts
        #Loop
        for i in range(n_Clusters):
            end_index = start_index  + clusters_size[i] 
            clusters_start_end_indexes[i] = [start_index,end_index] 
            start_index = end_index
        #for i in range(n_Clusters):       
            #print("Cluster [",i,"] : start_index : ",clusters_start_end_indexes[i][0]," & end_index ",clusters_start_end_indexes[i][1])
                
        
        #Saving for loading it in retrive func. 
        np.save("./" + self._file_path + "/data_points", sorted_data_sample)  
        #Saving for loading it in retrive func. 
        np.save("./" + self._file_path + "/data_points_indexes", sorted_data_sample_indexes)         
        #Saving Data samples Centiodes for loading it in retrive func.
        np.save("./" + self._file_path + "/Cluster_Centriods", kmeans.cluster_centers_ )
        #Saving Start & End indexes in each Centiode for loading it in retrive func.
        np.save("./" + self._file_path + "/Cluster_Indexes", clusters_start_end_indexes)


## These are the functions for running and reporting

In [8]:
import numpy as np
#from vec_db import VecDB
import time
from dataclasses import dataclass
from typing import List
#from memory_profiler import memory_usage
import gc

@dataclass
class Result:
    run_time: float
    top_k: int
    db_ids: List[int]
    actual_ids: List[int]

results = []
to_print_arr = []

def run_queries(db, query, top_k, actual_ids, num_runs):
    global results
    results = []
    for _ in range(num_runs):
        tic = time.time()
        db_ids = db.retrive(query, top_k)
        toc = time.time()
        run_time = toc - tic
        results.append(Result(run_time, top_k, db_ids, actual_ids))
    return results

def memory_usage_run_queries(args):
    global results
    # This part is added to calcauate the RAM usage
    #mem_before = max(memory_usage())
    #mem = memory_usage(proc=(run_queries, args, {}), interval = 1e-3)
    return results, 5 #max(mem) - mem_before

def evaluate_result(results: List[Result]):
    # scores are negative. So getting 0 is the best score.
    scores = []
    run_time = []
    for res in results:
        run_time.append(res.run_time)
        # case for retireving number not equal to top_k, socre will be the lowest
        if len(set(res.db_ids)) != res.top_k or len(res.db_ids) != res.top_k:
            scores.append( -1 * len(res.actual_ids) * res.top_k)
            continue
        score = 0
        for id in res.db_ids:
            try:
                ind = res.actual_ids.index(id)
                if ind > res.top_k * 3:
                    score -= ind
            except:
                score -= len(res.actual_ids)
        scores.append(score)

    return sum(scores) / len(scores), sum(run_time) / len(run_time)

def get_actual_ids_first_k(actual_sorted_ids, k):
    return [id for id in actual_sorted_ids if id < k]

## This to generate 10K database and the query using the seed numbers

In [9]:
QUERY_SEED_NUMBER = 10
DB_SEED_NUMBER = 20

In [11]:
rng = np.random.default_rng(DB_SEED_NUMBER)
vectors = rng.random((10000, 70), dtype=np.float32)
#print ('vectors are : ',vectors)
rng = np.random.default_rng(QUERY_SEED_NUMBER)
query = rng.random((1, 70), dtype=np.float32)
print ('query are : ',query)
actual_sorted_ids_10k = np.argsort(vectors.dot(query.T).T / (np.linalg.norm(vectors, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]

query are :  [[0.7765375  0.9560017  0.2640193  0.20768178 0.79258186 0.82844484
  0.51472414 0.1492821  0.8328704  0.51280457 0.15334606 0.13591957
  0.41092372 0.6890364  0.4036622  0.8417477  0.00812364 0.42550898
  0.52419096 0.956926   0.23533827 0.8253329  0.07183987 0.3382153
  0.74872607 0.57576054 0.93872505 0.75330186 0.9143402  0.8271039
  0.1357916  0.9334384  0.8445934  0.14499468 0.9784896  0.7455802
  0.31431204 0.13935137 0.3885808  0.9065287  0.78565687 0.22611439
  0.49179822 0.8532397  0.64099747 0.3063178  0.11379027 0.96983033
  0.2343331  0.5178342  0.6922639  0.32247454 0.5165536  0.2824335
  0.8366852  0.60586494 0.17676568 0.33376443 0.68798494 0.67864877
  0.31203574 0.15442502 0.14845031 0.24977547 0.7895685  0.8698942
  0.3430732  0.6003678  0.49958014 0.26198304]]


## Open new DB add 10K then retrieve and evaluate. Then add another 90K (total 100K) then retrieve and evaluate

In [22]:
top_k =10
db = VecDB(file_path = 'PATH_DB_10K', new_db = False)
records_dict = [{"id": i, "embed": list(row)} for i, row in enumerate(vectors)]
db.insert_records(records_dict)
res = run_queries(db, query, 10 , actual_sorted_ids_10k, 1) # one run to make everything fresh and loaded
########################TODO : comment this code
final_scores=[]
actual_sorted_topk= [actual_sorted_ids_1k[id] for id in range(len(actual_sorted_ids_1k)) if id < 10]
print ('actual_sorted_ids is : ', actual_sorted_topk)
for id in range(len(actual_sorted_topk)):
            # Access the current centroid using its index
            point_id = actual_sorted_topk[id]
            #calculating cosine simularity
            score = db._cal_score(query, vectors[point_id])
            final_scores.append(score)
print ('actual_sorted_points score is : ', final_scores)
########################
res, mem = memory_usage_run_queries((db, query, 10, actual_sorted_ids_10k, 5)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"10K\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)



actual_sorted_ids is :  [6233, 3332, 2038, 6587, 8367, 1361, 7600, 9925, 5065, 9640]
actual_sorted_points score is :  [array([0.8718706], dtype=float32), array([0.8687765], dtype=float32), array([0.8622327], dtype=float32), array([0.8602855], dtype=float32), array([0.8589762], dtype=float32), array([0.8583203], dtype=float32), array([0.8576241], dtype=float32), array([0.8573488], dtype=float32), array([0.85469675], dtype=float32), array([0.8544617], dtype=float32)]
10K	score	0.0	time	0.13	RAM	5.00 MB


## Remove exsiting varaibles to empty some RAM

In [9]:
del vectors
del query
del actual_sorted_ids_10k
del records_dict
del db
gc.collect()

214

## This code to generate 20M database. The seed (50) will not be changed. Create the same DB and prepare it's files indexes and every related file.
Note at the submission I'll not run the insert records.
The query istelf will be changed at submissions day but not the DB

In [6]:
vectors_20M = np.load("./PATH_DB_1K/data_points.npy",allow_pickle=True)
rng = np.random.default_rng(QUERY_SEED_NUMBER)
query = rng.random((1, 70), dtype=np.float32)

actual_sorted_ids_20m = np.argsort(vectors_20M.dot(query.T).T / (np.linalg.norm(vectors_20M, axis=1) * np.linalg.norm(query)), axis= 1).squeeze().tolist()[::-1]

In [11]:
vectors = np.load("./PATH_DB_100K/data_points.npy",allow_pickle=True)
db = VecDB(file_path = 'PATH_DB_100K', new_db = False)
db._build_index(vectors)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**5)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"100K\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)
del db
del vectors
gc.collect()

actual_sorted_ids is :  [664, 827, 458, 515, 535, 52, 483, 377, 855, 513, 795, 719, 822, 467, 581, 141, 730, 892, 994, 529, 405, 115, 934, 534, 488, 251, 91, 292, 619, 58, 555, 840, 159, 713, 217, 110, 13, 891, 376, 163, 202, 392, 143, 209, 743, 519, 576, 952, 375, 194, 224, 321, 707, 709, 423, 717, 306, 37, 51, 353, 894, 554, 567, 104, 318, 290, 308, 6, 484, 955, 670, 566, 965, 939, 204, 157, 736, 28, 927, 383, 245, 784, 259, 114, 26, 954, 695, 856, 942, 356, 388, 191, 816, 541, 689, 112, 814, 476, 510, 492, 285, 489, 958, 165, 626, 585, 734, 629, 304, 959, 548, 378, 505, 319, 578, 770, 485, 203, 428, 417, 787, 910, 925, 986, 373, 681, 539, 120, 477, 727, 531, 545, 956, 179, 244, 838, 466, 479, 399, 917, 989, 686, 269, 355, 270, 831, 644, 186, 690, 741, 228, 365, 731, 382, 800, 460, 845, 507, 449, 516, 268, 389, 973, 39, 282, 338, 599, 733, 915, 782, 164, 288, 41, 500, 137, 339, 899, 597, 662, 832, 151, 456, 705, 763, 133, 808, 860, 553, 693, 841, 183, 27, 761, 604, 936, 187, 879, 825

14

In [13]:
vectors = np.load("./PATH_DB_1M/data_points.npy",allow_pickle=True)
db = VecDB(file_path = 'PATH_DB_1M', new_db = False)
db._build_index(vectors)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"1M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)
del db
del vectors
gc.collect()

closest Centriod is :  292
closest Points_indexes is :  [277801, 572046, 505656, 30887, 652664]
closest Points score is :  [array([0.889826], dtype=float32), array([0.8799153], dtype=float32), array([0.877457], dtype=float32), array([0.8762298], dtype=float32), array([0.87601733], dtype=float32)]
1M	score	-5000.0	time	1.14	RAM	5.00 MB


0

In [8]:
vectors = np.load("./PATH_DB_5M/data_points.npy",allow_pickle=True)
db = VecDB(file_path = 'PATH_DB_5M', new_db = False)
db._build_index(vectors)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*5)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"5M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)
del db
del vectors
gc.collect()

closest Centriod is :  472
closest Points_indexes is :  [2354940, 4520447, 3130298, 884893, 2444666]
closest Points score is :  [array([0.87778115], dtype=float32), array([0.8748292], dtype=float32), array([0.87368125], dtype=float32), array([0.871892], dtype=float32), array([0.87081224], dtype=float32)]
5M	score	-5000.0	time	0.90	RAM	5.00 MB


13

In [9]:
vectors = np.load("./PATH_DB_10M/data_points.npy",allow_pickle=True)
db = VecDB(file_path = 'PATH_DB_10M', new_db = False)
db._build_index(vectors)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*10)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"10M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)
del db
del vectors
gc.collect()

closest Centriod is :  621
closest Points_indexes is :  [436098, 7623530, 8802913, 8536416, 1998541]
closest Points score is :  [array([0.88028765], dtype=float32), array([0.8796703], dtype=float32), array([0.8786013], dtype=float32), array([0.87832755], dtype=float32), array([0.8768755], dtype=float32)]
10M	score	-5000.0	time	3.88	RAM	5.00 MB


0

In [ ]:
vectors = np.load("./PATH_DB_15M/data_points.npy",allow_pickle=True)
db = VecDB(file_path = 'PATH_DB_15M', new_db = False)
db._build_index(vectors)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*15)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"15M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)
del db
del vectors
gc.collect()

In [ ]:
vectors = np.load("./PATH_DB_20M/data_points.npy",allow_pickle=True)
db = VecDB(file_path = 'PATH_DB_20M', new_db = False)
db._build_index(vectors)
actual_ids = get_actual_ids_first_k(actual_sorted_ids_20m, 10**6*20)
res = run_queries(db, query, 5, actual_ids, 1)  # one run to make everything fresh and loaded
res, mem = memory_usage_run_queries((db, query, 5, actual_ids, 3)) # actual runs to compute time, and memory
eval = evaluate_result(res)
to_print = f"20M\tscore\t{eval[0]}\ttime\t{eval[1]:.2f}\tRAM\t{mem:.2f} MB"
to_print_arr.append(to_print)
print(to_print)
del db
del vectors
gc.collect()